In [2]:
%pylab inline
#Autoreload so changing py files works
%load_ext autoreload
%autoreload 2
from Util import *
from superpoint import *

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
inputpath = 0
resize = False#(800, 600)
skip = 1
buffer = 5
max_frames = 99999
show = True
flip = False


extractor = SIFTFeatureExtractor()
matcher = BFMatcherHomographyGenerator()

def read():
    ret, frame = cap.read()
    if flip and frame.shape[0] < frame.shape[1]:
        frame = cv2.transpose(frame)
        frame = cv2.flip(frame, 1)
    if resize:
        frame = cv2.resize(frame, resize)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    color = frame#cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return color, gray

frames = []
cap = cv2.VideoCapture(inputpath)
correctedFrames = []
count = 0
H = np.array([[1,0,0],[0,1,0],[0,0,1]])

#Read first frame
color1, gray1 = read()
correctedFrames.append(color1)
kps1, desc1 = extractor.extractFeatures(gray1)
frames.append(gray1)

while cap.isOpened():
    #Read in next frame
    color2, gray2 = read()
    count = count + 1
    #Allow frame skipping
    if count > max_frames: break
    if count % skip == 0:
        if len(frames) < buffer:
            frames.append(gray2)
            kps2, desc2 = extractor.extractFeatures(gray2)
            HG = matcher.findHomography(kps2, desc2, kps1, desc1)
            H = H @ HG
            corrected = cv2.warpPerspective(color2, H, (color2.shape[1], color2.shape[0]))
            kps1 = kps2
            desc1 = desc2
        else:
            del frames[0]
            frames.append(gray2)
            H = np.array([[1,0,0],[0,1,0],[0,0,1]])
            kps1, desc1 = extractor.extractFeatures(frames[0])
            for i in range(1, len(frames)):
                kps2, desc2 = extractor.extractFeatures(frames[i])
                HG = matcher.findHomography(kps2, desc2, kps1, desc1)
                H = H @ HG
                kps1 = kps2
                desc1 = desc2
            corrected = cv2.warpPerspective(color2, H, (color2.shape[1], color2.shape[0]))
        
        correctedFrames.append(corrected)
        
        if show:
            cv2.imshow('frame', corrected)
    #Press space to end
    if cv2.waitKey(32) == 32:
        break
    
cap.release()